# 1 Carga de librerías

In [778]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
from scipy import stats as stats
from scipy.stats import ttest_ind
from scipy.stats import levene

# 2. Carga de dataset

In [779]:
# Uso de read_csv para leer los datos CSV, la "r" es para evitar los caracteres de escape
df = pd.read_csv(r'D:\Data analyst\datasets\games.csv')

In [780]:
#Uso de info para ver datos relevantes de los datos
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16715 entries, 0 to 16714
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Name             16713 non-null  object 
 1   Platform         16715 non-null  object 
 2   Year_of_Release  16446 non-null  float64
 3   Genre            16713 non-null  object 
 4   NA_sales         16715 non-null  float64
 5   EU_sales         16715 non-null  float64
 6   JP_sales         16715 non-null  float64
 7   Other_sales      16715 non-null  float64
 8   Critic_Score     8137 non-null   float64
 9   User_Score       10014 non-null  object 
 10  Rating           9949 non-null   object 
dtypes: float64(6), object(5)
memory usage: 1.4+ MB


Comentarios:
- Nomenclatura inadecuada de las columnas, se procede a usar minúsculas y snake_case
- Solo presentamos 2 tipos de datos, objeto y flotantes, se indagará para hacer la transformación al dato correcto.
- Encontramos valores nulos, se indagará para tomar una buena decisión de su tratamiento.

In [781]:
#Uso de head para ver rápidamente los datos
df.head(5)

,Name,Platform,Year_of_Release,Genre,NA_sales,EU_sales,JP_sales,Other_sales,Critic_Score,User_Score,Rating
0,Wii Sports,Wii,2006.0,Sports,41.36,28.96,3.77,8.45,76.0,8,E
1,Super Mario Bros.,NES,1985.0,Platform,29.08,3.58,6.81,0.77,NaN,NaN,NaN
2,Mario Kart Wii,Wii,2008.0,Racing,15.68,12.76,3.79,3.29,82.0,8.3,E
3,Wii Sports Resort,Wii,2009.0,Sports,15.61,10.93,3.28,2.95,80.0,8,E
4,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,11.27,8.89,10.22,1.00,NaN,NaN,NaN


# 3 Preparación de datos

### Modificación de columnas

In [782]:
#Ciclo for que itera sobre cada columna, aplicandole el método lower para volver a minúsculas
new_names = []
for old_names in df.columns:
    new_col_names = old_names.lower()
    new_names.append(new_col_names)

df.columns = new_names

df.rename(columns={'year_of_release':'date_of_release'},inplace=True)

df['date_of_release'] = pd.to_datetime(df['date_of_release'])#cambio de dato de str a datetime

df.insert(3,'year',df['date_of_release'].dt.year) #creación de nueva columna solo con el año

df.columns

Index(['name', 'platform', 'date_of_release', 'year', 'genre', 'na_sales',
       'eu_sales', 'jp_sales', 'other_sales', 'critic_score', 'user_score',
       'rating'],
      dtype='object')

### Evaluación y tratamiento de ausentes

In [783]:
#Suma de valores nulos
nans = df.isna().sum()
total_nans = nans.sum()
print(f'El dataframe tiene \033[4m\033[1m{total_nans}\033[0m\033[0m valores nulos')

El dataframe tiene 22587 valores nulos


In [784]:
#Análisis de las columnas con nans
nans

name                  2
platform              0
date_of_release     269
year                269
genre                 2
na_sales              0
eu_sales              0
jp_sales              0
other_sales           0
critic_score       8578
user_score         6701
rating             6766
dtype: int64

In [785]:
#Impresión de columnas con valores nulos
cols_with_nans = df.isna().any() #any evalua qué elemento cumple la condición de tener nulos con isna
list_cols_nans = df.loc[:, cols_with_nans].columns.to_list() 
'''
Selecciona las columnas con nulos --> df.loc[:,cols_with_nans],
Recupera solo el nombre de columna  --> .columns,
Vuelve a lista para imprimir --> .to_list()
'''
print(f'Columnas con valores nulos {list_cols_nans}')

Columnas con valores nulos ['name', 'date_of_release', 'year', 'genre', 'critic_score', 'user_score', 'rating']


In [788]:
#Eliminación de filas con nombre y género nulo para futuro análisis
df.dropna(subset=['name','genre'],inplace=True)
df.fillna({'date_of_release':'unknown'},inplace=True)
df.fillna({'year':'unknown'}, inplace=True)
df.isna().sum()

name                  0
platform              0
date_of_release       0
year                  0
genre                 0
na_sales              0
eu_sales              0
jp_sales              0
other_sales           0
critic_score       8576
user_score         6699
rating             6764
dtype: int64

#### Siglo de lanzamiento

In [787]:
#Filtro por año de lanzamiento, condición: Juegos lanzados antes del 2000 o siglo 21
century_20th_games = df[df['year'] < 2000].reset_index()
nans_20th = century_20th_games.isna().sum()
total_20th_nans = nans_20th.sum()
print(f'El dataframe de los juegos lanzados antes del 2000 tiene \033[4m\033[1m{total_20th_nans}\033[0m\033[0m valores nulos, \nesto representa el {(total_20th_nans*100)/total_nans:.2f}% del total de nulos.')

TypeError: '<' not supported between instances of 'str' and 'int'

In [ ]:
#Filtro para encontrar los nulos de juegos lanzados en el siglo 21
century_21th_games = df[df['year'] >= 2000].reset_index()
nans_21th = century_21th_games.isna().sum()
total_21th_nans = nans_21th.sum()
print(f'El dataframe de los juegos lanzados en el siglo 21 tiene \033[4m\033[1m{total_21th_nans}\033[0m\033[0m valores nulos, \nesto representa el {(total_21th_nans*100)/total_nans:.2f}% del total de nulos.')

El dataframe de los juegos lanzados en el siglo 21 tiene 0 valores nulos, 
esto representa el 0.00% del total de nulos.


In [ ]:
#Creación de listas con valores únicos
platforms = century_21th_games['platform'].unique().tolist()
years = century_21th_games['year'].unique().tolist()
genres = century_21th_games['genre'].unique().tolist()

**Comentario:**

Si analizamos el la tabla de columnas con sus nulos obtenida observamos que estos juegos carecen de puntuación referente a crítica, crítica de usuarios, clasificación del juego, nombre y género, esto puede deberse por el año, la falta de internet, el acceso a los consumidores de esos años o incluso por la obsolecencia de la consola para jugarlos.

**Decisión de nulos:**

Dependiendo el análisis, se podrá tomar en cuenta estos datos. Lista de casos donde se usarán:

- Cálculos de jugadores en las diferentes regiones.
- Análisis por género, año de lanzamiento y plataforma.

Casos en donde no se podrá considerar estos datos:

- Cálculos por rating, puntuación y por puntuación de usuarios.

De momento se ha decidido **<u>no eliminar</u>** estos nulos.

##### Plataforma

In [ ]:
#Ciclo for que indaga los nulos por plataforma
platforms_nulls = []
for platform in platforms:
    platform_filter = compare_df[compare_df['platform'].isin([platform])]
    platforms_nans = platform_filter.isna().sum().sum()
    platforms_nulls.append(platforms_nans)

platform_data = list(zip(platforms, platforms_nulls))
df_platform = pd.DataFrame(platform_data,columns=['platform','nans'])
df_platform.head()

,platform,nans


##### Año

In [ ]:
#Ciclo for que indaga los nulos por año
year_nulls = []
for year in years:
    year_filter = compare_df[compare_df['year'].isin([year])]
    year_nans = year_filter.isna().sum().sum()
    year_nulls.append(year_nans)
    
year_data = list(zip(years, year_nulls))
df_year = pd.DataFrame(year_data,columns=['year','nans'])
df_year.head()

,year,nans


##### Género

In [ ]:
#Ciclo for que indaga los nulos por genero
genre_nulls = []
for genre in genres:
    genre_filter = compare_df[compare_df['genre'].isin([genre])]
    genre_nans = genre_filter.isna().sum().sum()
    genre_nulls.append(genre_nans)
    
genre_data = list(zip(genres, genre_nulls))
df_genre = pd.DataFrame(genre_data,columns=['genre','nans'])
df_genre.head()

,genre,nans


**Decisión sobre nulos**

Hemos descubierto que no depende de la plataforma, género o año de lanzamiento, los nulos dependen de otras variables.

De momento, se ha decidido **<u>no eliminar</u>** estos nulos.

#### Ventas por Región

In [ ]:
eu_sales = century_21th_games[(century_21th_games['eu_sales']==0.0) & (century_21th_games['na_sales']>0.0) & (century_21th_games['jp_sales']>0.0) & (century_21th_games['other_sales']>0.0)].reset_index()
total_eu_nans = eu_sales.isna().sum().sum()
total_eu_nans

np.int64(0)

In [ ]:
na_sales = century_21th_games[(century_21th_games['na_sales']==0.0) & (century_21th_games['eu_sales']>0.0) & (century_21th_games['jp_sales']>0.0) & (century_21th_games['other_sales']>0.0)].reset_index()
total_na_nans = na_sales.isna().sum().sum()
total_na_nans

np.int64(0)

In [ ]:
jp_sales = century_21th_games[(century_21th_games['jp_sales']==0.0) & (century_21th_games['eu_sales']>0.0) & (century_21th_games['na_sales']>0.0) & (century_21th_games['other_sales']>0.0)].reset_index()
total_jp_nans = jp_sales.isna().sum().sum()
total_jp_nans

np.int64(0)

In [ ]:
other_sales = century_21th_games[(century_21th_games['other_sales']==0.0) & (century_21th_games['eu_sales']>0.0) & (century_21th_games['jp_sales']>0.0) & (century_21th_games['na_sales']>0.0)].reset_index()
total_other_nans = other_sales.isna().sum().sum()
total_other_nans

np.int64(0)

**Comentario**

Hemos descubierto que los juegos que no tuvieron ventas en japón tienen una gran cantidad de nulos.

In [ ]:
all_sales = century_21th_games[(century_21th_games['other_sales']==0.0) | (century_21th_games['eu_sales']==0.0) | (century_21th_games['jp_sales']==0.0) | (century_21th_games['na_sales']==0.0)].reset_index()
total_all_nans = all_sales.isna().sum().sum()
total_all_nans

np.int64(0)

**Comentario**

Al parecer, los valores nulos dependen de las ventas, éstos pudieron ser por falta de ventas en una o más regiones.

### Cálculo de ventas

In [ ]:
df['total_sales'] = df['na_sales'] + df['eu_sales'] + df['jp_sales'] + df['other_sales']
df.head()

,name,platform,date_of_release,year,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating,total_sales
0,Wii Sports,Wii,1970-01-01 00:00:00.000002006,1970.0,Sports,41.36,28.96,3.77,8.45,76.0,8,E,82.54
1,Super Mario Bros.,NES,1970-01-01 00:00:00.000001985,1970.0,Platform,29.08,3.58,6.81,0.77,NaN,NaN,NaN,40.24
2,Mario Kart Wii,Wii,1970-01-01 00:00:00.000002008,1970.0,Racing,15.68,12.76,3.79,3.29,82.0,8.3,E,35.52
3,Wii Sports Resort,Wii,1970-01-01 00:00:00.000002009,1970.0,Sports,15.61,10.93,3.28,2.95,80.0,8,E,32.77
4,Pokemon Red/Pokemon Blue,GB,1970-01-01 00:00:00.000001996,1970.0,Role-Playing,11.27,8.89,10.22,1.00,NaN,NaN,NaN,31.38
